In [102]:
import torch
from transformers import AutoTokenizer
import json
import requests
from transformers import LayoutLMModel
from pytorch_lightning import LightningModule
import torch.nn as nn
from infer.model.loss import BboxLoss
from infer.model.spade_model import RelationTagger
import json

# jsonfile =  json.load(open('./notebook/json_file/cccd_test.jsonl')) 

tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")
def get_bbox(jsonl_file):
    w = jsonl_file['img_sz']['width']
    h = jsonl_file['img_sz']['height']
    bboxes = [[ int(x[0][0]*1000/w),int(x[0][1]*1000/h) ,int(x[2][0]*1000/w),int(x[2][1]*1000/h)] for x in jsonl_file['coord']]
    # return  torch.tensor(bboxes)
    return bboxes

In [103]:
from dataModel.datamd_v2 import DpDataSet
import numpy as np
data = DpDataSet('./notebook/json_file/cccd_test.jsonl')

data[0]['maps_tensor'].shape, 

i = 0
for item in data[0]['maps']:
    for idx in item: 
        i = i + 1
i, data[0]['maps_tensor'].shape

(188, torch.Size([1, 188]))

In [104]:
# m = np.array(maps.cpu())
# m = m.tolist()
# box_index =  np.unique(m).tolist()
# [[item]*m.count(item) for item in box_index]
# input_ids.shape
from functools import lru_cache
@lru_cache
def load_model():

    return LayoutLMModel.from_pretrained(
                "microsoft/layoutlm-base-uncased", local_files_only=True)


In [105]:
from torch import Tensor
model =  load_model()
# maps
input_ids = data[0]['input_ids']
bbox = data[0]['bbox']
attention_mask = data[0]['attention_mask']
token_type_ids = data[0]['token_type_ids']
maps = data[0]['maps']
maps_tensor = data[0]['maps_tensor']

Some weights of the model checkpoint at microsoft/layoutlm-base-uncased were not used when initializing LayoutLMModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing LayoutLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [239]:
outputs = model(input_ids=input_ids,bbox=bbox,attention_mask=attention_mask,token_type_ids=token_type_ids)
outputs.last_hidden_state.shape

reduce_size = 256

ln = nn.Linear(model.config.hidden_size, reduce_size)

last_hidden_state = ln(outputs.last_hidden_state)
last_hidden_state.shape

zeros_temp = torch.zeros(reduce_size)

last_hidden_state

tensor([[[ 0.2074,  0.4719, -0.0585,  ...,  0.0695, -0.2073, -0.5689],
         [-0.1731,  0.2456, -0.0784,  ..., -0.1674,  0.1176, -0.3298],
         [ 0.4754,  0.1427, -0.2165,  ...,  0.1982, -0.1680, -0.2255],
         ...,
         [ 0.2280,  0.2522, -0.2653,  ...,  0.1527,  0.2269,  0.0562],
         [-0.0099,  0.4010, -0.3135,  ..., -0.1487,  0.1672, -0.1883],
         [-0.0500,  0.2073,  0.3541,  ..., -0.3969, -0.1453, -0.2007]]],
       grad_fn=<AddBackward0>)

In [145]:

# def tranfer_maps(maps: Tensor):
#     m = []
#     maps = torch.squeeze(maps, 0)
#     uniq, count= maps.unique(return_counts=True)
#     for i in range(len(count)):
#         m.append(torch.stack([uniq[i]]*count[i]))
#     return m


def reduce_shape_old(last_hidden_state, maps):
        device = last_hidden_state.device
        reduce = []
        for g_token in maps:
            ten = zeros_temp.to(device)
            for ele in g_token:
                # print(g_token[0],'-----')
                ten += last_hidden_state[0][g_token[0]]
                # print(ele,'----')
            ten = ten/len(g_token)
            # print(len(g_token))
            reduce.append(ten)
        reduce = torch.stack(reduce).to(device)
        return reduce



reduce_old = reduce_shape_old(last_hidden_state, maps)


# tranfer_maps(maps)
# maps = 
# reduce_shape(last_hidden_state, tranfer_maps(maps) ) 


In [218]:

def reduce_shape(last_hidden_state, maps: Tensor):
    maps = torch.squeeze(maps, 0)
    uniq, count= maps.unique(return_counts=True)
    device = last_hidden_state.device
    reduce = []
    for idx,  g_token in enumerate (uniq):
        ten = zeros_temp.to(device)
        for j in range (count[idx]):
            # print(g_token.numpy(),'-----')
            ten += last_hidden_state[0][g_token.numpy()]
        # print(g_token, idx)
        ten = ten/count.detach().numpy()[idx]
        # print(count.detach().numpy()[idx])
        reduce.append(ten)
    reduce = torch.stack(reduce).to(device)
    return reduce
reduce = reduce_shape(last_hidden_state, maps_tensor)
# # maps

# with torch.no_grad():
#   b =  torch.mean(reduce -reduce_old)
#   print(b)

reduce

tensor([[ 8020.1729, -5490.8179,   162.1636,  ..., -4435.5386,  6646.5469,
          5353.5430],
        [ 4010.5117, -2745.9504,    81.3024,  ..., -2217.8000,  3323.6831,
          2676.8198],
        [ 8021.1714, -5492.0972,   162.6142,  ..., -4436.2832,  6647.3828,
          5353.8818],
        ...,
        [ 8083.3086, -5533.5264,   163.7679,  ..., -4470.3042,  6698.4453,
          5396.2227],
        [ 8083.7129, -5533.8901,   163.7538,  ..., -4470.7573,  6698.7896,
          5396.1689],
        [ 8084.0513, -5534.0591,   163.5114,  ..., -4470.8013,  6698.9219,
          5396.2764]], grad_fn=<StackBackward0>)

In [148]:
def tranfer_maps(maps: Tensor):
    m = []
    maps = torch.squeeze(maps, 0)
    uniq, count= maps.unique(return_counts=True)
    for i in range(len(count)):
        m.append(torch.stack([uniq[i]]*count[i]).tolist())
    return m



a = reduce_shape_old(last_hidden_state, maps) - reduce_shape_old(last_hidden_state,maps )
with torch.no_grad():
    
    print(torch.mean(a))

# tranfer_maps(maps_tensor), maps

tensor(-0.6993)


In [ ]:

# %reload_ext autoreload
from models.layout_parsing import LitBaseParsing
%load_ext autoreload
%autoreload 2
from models.base import  LitLayoutParsing
model = LitBaseParsing()

In [ ]:
# totalGraph =  model(input_ids, attention_mask, token_type_ids, bbox,maps)

# totalGraph.shape
dummy_input = torch.zeros(3,126)
dummy_input[1]

In [ ]:
# dummy_input = torch.zeros_like(input_ids).cuda()
# dummy_boxes = torch.zeros_like(bbox).cuda()
# dummy_maps = torch.zeros_like(maps).cuda()
import torch
dummy_input = torch.zeros(1,196).cuda()
dummy_boxes = torch.zeros(1,196,4).cuda()
dummy_maps = torch.zeros(1,196).cuda()
# totalGraph[0].shape
# S = totalGraph[0].unsqueeze(0)
# G = totalGraph[1].unsqueeze(0)
# dummy_maps.shape
# dummy_maps.unsqueeze(0).shape

In [1]:
import onnxruntime as ort
import numpy as np


providers = [
            # 'TensorrtExecutionProvider',
                     'CUDAExecutionProvider',
                     'CPUExecutionProvider']

ort_sess = ort.InferenceSession(
                                path_or_bytes = 'resources/onnx/LitLP_06-03-2023_18-46-25_fp32.onnx'
                                , providers= providers)

In [5]:
input_name = ort_sess.get_inputs()
# output_name = ort_sess.get_outputs()[0].name
input_name[3].name


# outputs = ort_sess.run(None, 
#                         {
#                             'input_1': input_ids.numpy()
#                             'input_2': input_ids.numpy()
#                             'input_3': input_ids.numpy()
#                             'input_4': input_ids.numpy()
#                             'input_5': input_ids.numpy()
#                         }
#                         )


'bbox'

In [ ]:
import torch




a = torch.tensor([1, 2, 2, 1,                                                                        
1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 7,                          
        1, 1, 1, 2, 1, 2, 2, 5, 2, 3, 1, 3, 2, 1, 1, 2, 1, 3, 3, 2, 2, 2, 3, 2,      
        1, 1, 2, 2, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 6, 1, 1, 3, 1])

In [101]:
# 1/a[1].numpy()

# for i in range(a[1]):
f = 'hihi'
tr= f"dasudlasdjak{f} 'adas'dasoi'"
#     print(i)
tr

"dasudlasdjakhihi 'adas'dasoi'"